In [66]:
import numpy as np
from PIL import Image
from make_gts import color_map
import os
import re

In [57]:
cmap = color_map('fundus')

In [28]:
def compute_padding(length):
    length_padding = 512-length
    left_bottom_padding = np.around(length_padding/2).astype(np.uint8)
    right_top_padding = left_bottom_padding
    if not length_padding % 2:
        right_top_padding += 1
    return left_bottom_padding,right_top_padding

def crop(img_path,mask_path):
    mask_arr = np.array(Image.open(mask_path))
    img_arr = np.array(Image.open(img_path))
    mask_lt0= mask_arr > 0
    mask_lt0_indexs  = np.where(mask_lt0)
    width_idx,height_idx = mask_lt0_indexs[0],mask_lt0_indexs[1]
    top,bottom = width_idx.min(),width_idx.max()
    left,right = height_idx.min(),height_idx.max()

    width = right-left
    height = bottom-top
    left_padding,right_padding = compute_padding(width)
    bottom_padding,top_padding = compute_padding(height)
    # 开始裁剪图片
    new_left = left-left_padding
    new_right = right+right_padding-1
    new_bottom = bottom+bottom_padding-1
    new_top = top-top_padding
    cropped_mask_arr = mask_arr[new_top:new_bottom,new_left:new_right]
    cropped_img_arr = img_arr[new_top:new_bottom,new_left:new_right]

    cropped_img = Image.fromarray(cropped_img_arr)
    cropped_mask = Image.fromarray(cropped_mask_arr,mode='P')
    cropped_mask.putpalette(cmap)
    return cropped_img,cropped_mask

In [65]:
image_root = '/root/autodl-tmp/data/SEG'
for root,dirs,files in os.walk(image_root):
    pattern = re.compile(r'.*\..*')
    if pattern.match(files):
        for file in files:
            path = os.path.join(root,file)
            print(os.path.basename(path))


[0 1 2]
(512, 512)
